# non-linear pde- prey predator model - lotka volterra
 dx/dt = ax - bxy

 dy/dt = cxy - dy              initial cond : x= 2000, y=6000, find y and x

 a = 0.1

 b = 0.4

 c = 0.1
 
 d = 0.4

In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:

class Net_x(nn.Module):
    def __init__(self):
        super(Net_x, self).__init__()
        self.hidden_layer1 = nn.Linear(1,5)
        self.hidden_layer2 = nn.Linear(5,5)
        self.hidden_layer3 = nn.Linear(5,5)
        self.hidden_layer4 = nn.Linear(5,5)
        self.hidden_layer5 = nn.Linear(5,5)
        self.output_layer = nn.Linear(5,1)

    def forward(self,t):
        inputs = torch.cat([t],axis=1) # combined two arrays of 1 columns each to one array of 2 columns
        layer1_out = torch.sigmoid(self.hidden_layer1(inputs))
        layer2_out = torch.sigmoid(self.hidden_layer2(layer1_out))
        layer3_out = torch.sigmoid(self.hidden_layer3(layer2_out))
        layer4_out = torch.sigmoid(self.hidden_layer4(layer3_out))
        layer5_out = torch.sigmoid(self.hidden_layer5(layer4_out))
        output = self.output_layer(layer5_out) ## For regression, no activation is used in output layer
        return output

In [3]:

class Net_y(nn.Module):
    def __init__(self):
        super(Net_y, self).__init__()
        self.hidden_layer1 = nn.Linear(1,5)
        self.hidden_layer2 = nn.Linear(5,5)
        self.hidden_layer3 = nn.Linear(5,5)
        self.hidden_layer4 = nn.Linear(5,5)
        self.hidden_layer5 = nn.Linear(5,5)
        self.output_layer = nn.Linear(5,1)

    def forward(self, t):
        inputs = torch.cat([t],axis=1) # combined two arrays of 1 columns each to one array of 2 columns
        layer1_out = torch.sigmoid(self.hidden_layer1(inputs))
        layer2_out = torch.sigmoid(self.hidden_layer2(layer1_out))
        layer3_out = torch.sigmoid(self.hidden_layer3(layer2_out))
        layer4_out = torch.sigmoid(self.hidden_layer4(layer3_out))
        layer5_out = torch.sigmoid(self.hidden_layer5(layer4_out))
        output = self.output_layer(layer5_out) ## For regression, no activation is used in output layer
        return output

In [4]:
### (2) Model
net_x = Net_x()
net_y = Net_y()
net_x = net_x.to(device)
net_y = net_y.to(device)
mse_cost_function = torch.nn.MSELoss() # Mean squared error
optimizer = torch.optim.Adam(list(net_x.parameters())+ list(net_y.parameters()))

In [5]:
## PDE as loss function. Thus would use the network which we call as u_theta
def f(t, net_x, net_y):
    u = net_x(t) # the dependent variable u is given by the network based on independent variables x,t
    v = net_y(t)
    
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    v_t = torch.autograd.grad(v.sum(), t, create_graph=True)[0]
    

    pde_1 = u_t - 0.1*u + 0.4*u*v
    pde_2 =v_t - 0.1*u*v + 0.4*v
    
    return pde_1, pde_2

In [6]:
# b.c ==> u(0) = 2000   and v(0) = 6000
t_bc = np.zeros((500,1))
u_bc = 2000*np.ones((500,1))
v_bc = 6000*np.ones((500,1))

In [7]:
u_bc

array([[2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [2000.],
       [

In [19]:
### (3) Training / Fitting
iterations = 199999
epoch = 0
loss =  torch.tensor(10000)
#previous_validation_loss = 99999999.0
#for epoch in range(iterations):
while True :
    epoch += 1
    optimizer.zero_grad() # to make the gradients zero
    
    # Loss based on boundary conditions
    #pt_x_bc = Variable(torch.Tensor(x_bc).float(), requires_grad=False).to(device)
    pt_t_bc = Variable(torch.Tensor(t_bc).float(), requires_grad=False).to(device)
    pt_u_bc = Variable(torch.Tensor(u_bc).float(), requires_grad=False).to(device)
    pt_v_bc = Variable(torch.Tensor(v_bc).float(), requires_grad=False).to(device)

    
    net_u_bc_out = net_x(pt_t_bc) # output of u(x,t)
    mse_u_bc = mse_cost_function(net_u_bc_out, pt_u_bc)

    net_v_bc_out = net_y(pt_t_bc) # output of u(x,t)
    mse_v_bc = mse_cost_function(net_v_bc_out, pt_v_bc)
    
    # Loss based on PDE
    #x_collocation = np.random.uniform(low=0.0, high=2.0, size=(500,1))
    t_collocation = np.random.uniform(low=0.0, high=1000.0, size=(500,1))
    all_zeros_1 = np.zeros((500,1))
    all_zeros_2 = np.zeros((500,1))
    
    
    #pt_x_collocation = Variable(torch.Tensor(x_collocation).float(), requires_grad=True).to(device)
    pt_t_collocation = Variable(torch.Tensor(t_collocation).float(), requires_grad=True).to(device)
    pt_all_zeros_1 = Variable(torch.Tensor(all_zeros_1).float(), requires_grad=False).to(device)
    pt_all_zeros_2 = Variable(torch.Tensor(all_zeros_1).float(), requires_grad=False).to(device)

    
    pde_1, pde_2 = f(pt_t_collocation, net_x, net_y) # output of the differential eqn
    mse_f_1 = mse_cost_function(pde_1, pt_all_zeros_1)
    mse_f_2 = mse_cost_function(pde_2, pt_all_zeros_2)
    
    # Combining the loss functions
    loss = mse_u_bc + mse_v_bc  + mse_f_1 + mse_f_2
    
    
    loss.backward() # This is for computing gradients using backward propagation
    optimizer.step() # This is equivalent to : theta_new = theta_old - alpha * derivative of J w.r.t theta

    with torch.autograd.no_grad():
    	print(epoch,"Traning Loss:",loss.data)

    if (loss.item() <= 0.00005):
                        break

        

1 Traning Loss: tensor(38199424.)
2 Traning Loss: tensor(38199320.)
3 Traning Loss: tensor(38199240.)
4 Traning Loss: tensor(38199152.)
5 Traning Loss: tensor(38199056.)
6 Traning Loss: tensor(38198976.)
7 Traning Loss: tensor(38198876.)
8 Traning Loss: tensor(38198792.)
9 Traning Loss: tensor(38198700.)
10 Traning Loss: tensor(38198616.)
11 Traning Loss: tensor(38198536.)
12 Traning Loss: tensor(38241080.)
13 Traning Loss: tensor(38198348.)
14 Traning Loss: tensor(38198260.)
15 Traning Loss: tensor(38198172.)
16 Traning Loss: tensor(38198092.)
17 Traning Loss: tensor(38198004.)
18 Traning Loss: tensor(38197908.)
19 Traning Loss: tensor(38197820.)
20 Traning Loss: tensor(38197732.)
21 Traning Loss: tensor(38197652.)
22 Traning Loss: tensor(38197540.)
23 Traning Loss: tensor(38197460.)
24 Traning Loss: tensor(38197372.)
25 Traning Loss: tensor(38197284.)
26 Traning Loss: tensor(38197196.)
27 Traning Loss: tensor(38197116.)
28 Traning Loss: tensor(38197016.)
29 Traning Loss: tensor(38196

KeyboardInterrupt: 

In [18]:
torch.tensor(10000).item()

10000